In [7]:
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, f1_score

def clean_text(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove URLs
        text = re.sub(r'http\S+', '', text)
        # Remove hashtags but keep the text
        text = re.sub(r'#', '', text)
        # Remove special characters
        text = re.sub(r'[^\w\s]', ' ', text)
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

def extract_topic(row, topics_list):
    text = str(row['Training Topic'])

    # Find which topic appears in the text
    for topic in topics_list:
        if topic.lower() in text.lower():
            return topic

    # checking hashtags for topic clues
    for topic in topics_list:
        # Create hashtag variations
        hashtag = f"#{topic.lower().replace(' ', '')}"
        hashtag2 = f"#{topic.lower().replace(' & ', '').replace(' ', '')}"
        if hashtag in text.lower() or hashtag2 in text.lower():
            return topic

    # common keywords
    keyword_mapping = {
        'corruption': 'Corruption',
        'election': 'Election',
        'vote': 'Election',
        'environment': 'Environment',
        'climate': 'Environment',
        'police': 'Law and Order',
        'crime': 'Law and Order',
        'women': 'Women Rights',
        'gender': 'Women Rights',
        'sport': 'Sports',
        'terrorism': 'Terrorism',
        'education': 'Education',
        'religion': 'Religion',
        'trade': 'Trade & Commodity Price',
        'politics': 'Politics',
        'minister': 'Politics',
        'government': 'Politics'
    }

    for keyword, topic in keyword_mapping.items():
        if keyword in text.lower():
            return topic

    # Default to the most common class if no match
    return 'Politics'  # Assuming Politics is the most common

def main():
    topics = [
        'Politics', 'Law and Order', 'Governance & Policy Reform', 'Sports',
        'Culture & Lifestyle', 'Corruption', 'International affairs', 'Election',
        'Environment', 'Natural Disaster', 'Terrorism', 'Mob Justice',
        'Women Rights', 'Islamic Fundamentalism', 'National Defence', 'Diplomacy',
        'Education', 'Religion', 'Trade & Commodity Price'
    ]

    # dataset load
    try:
        df = pd.read_excel('data_and_topics.xlsx', sheet_name='Training Dataset')
    except:
        print("Could not find the Excel file.")
        return

    print(f"Dataset shape: {df.shape}")
    print(df.head())

    # cleaning function apply
    df['cleaned_text'] = df['Context'].apply(clean_text)

    # extract the topic labels
    df['topic'] = df.apply(lambda row: extract_topic(row, topics), axis=1)

    # distribution of topics check
    print("\nTopic distribution:")
    topic_distribution = df['topic'].value_counts()
    print(topic_distribution)

    # Plot topic distribution
    plt.figure(figsize=(12, 6))
    topic_distribution.plot(kind='bar')
    plt.title('Topic Distribution')
    plt.xlabel('Topic')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('topic_distribution.png')
    plt.close()

    # split into features and labels
    X = df['cleaned_text']
    y = df['topic']

    #train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # STEP 1: BASE MODEL
    print("\n--- Building the Base Model ---")

    # Creating TF-IDF features
    tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    # training a basic Naive Bayes model
    clf = MultinomialNB()
    clf.fit(X_train_tfidf, y_train)

    # predict on the training data
    y_train_pred = clf.predict(X_train_tfidf)

    # Calculate training metrics
    train_precision = precision_score(y_train, y_train_pred, average='weighted')
    train_recall = recall_score(y_train, y_train_pred, average='weighted')
    train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    train_cm = confusion_matrix(y_train, y_train_pred)

    print("\nTraining Set Metrics:")
    print(f"Precision: {train_precision:.4f}")
    print(f"Recall: {train_recall:.4f}")
    print(f"F1 Score: {train_f1:.4f}")

    # predict on the test data
    y_test_pred = clf.predict(X_test_tfidf)

    # calculate test metrics
    test_precision = precision_score(y_test, y_test_pred, average='weighted')
    test_recall = recall_score(y_test, y_test_pred, average='weighted')
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    test_cm = confusion_matrix(y_test, y_test_pred)

    print("\nTest Set Metrics:")
    print(f"Precision: {test_precision:.4f}")
    print(f"Recall: {test_recall:.4f}")
    print(f"F1 Score: {test_f1:.4f}")

    # classification report
    print("\nClassification Report (Test Set):")
    print(classification_report(y_test, y_test_pred))

    # base model metrics for comparison
    base_metrics = {
        'train': {
            'precision': train_precision,
            'recall': train_recall,
            'f1': train_f1
        },
        'test': {
            'precision': test_precision,
            'recall': test_recall,
            'f1': test_f1
        }
    }

    # synthetic test set
    synthetic_data = {
        'Context': [
            "Bangladesh government introduces new anti-corruption measures",
            "Local sports team wins national championship",
            "New environmental protection law passed",
            "Women's rights activists protest for equal pay",
            "Election commission announces dates for upcoming vote",
            "Police arrest suspect in robbery case",
            "Religious leaders call for peace and harmony",
            "Trade relations with neighboring countries improve",
            "Defense minister announces new security policy",
            "Educational reforms to improve literacy rates"
        ],
        'True_Topic': [
            'Corruption',
            'Sports',
            'Environment',
            'Women Rights',
            'Election',
            'Law and Order',
            'Religion',
            'Trade & Commodity Price',
            'National Defence',
            'Education'
        ]
    }
    synthetic_df = pd.DataFrame(synthetic_data)
    synthetic_df['cleaned_text'] = synthetic_df['Context'].apply(clean_text)
    X_synthetic = synthetic_df['cleaned_text']
    y_synthetic = synthetic_df['True_Topic']

    # transform and predict on synthetic data
    X_synthetic_tfidf = tfidf_vectorizer.transform(X_synthetic)
    y_synthetic_pred = clf.predict(X_synthetic_tfidf)

    # Calculate metrics on synthetic test set
    synthetic_precision = precision_score(y_synthetic, y_synthetic_pred, average='weighted')
    synthetic_recall = recall_score(y_synthetic, y_synthetic_pred, average='weighted')
    synthetic_f1 = f1_score(y_synthetic, y_synthetic_pred, average='weighted')

    print("\nSynthetic Test Set Metrics:")
    print(f"Precision: {synthetic_precision:.4f}")
    print(f"Recall: {synthetic_recall:.4f}")
    print(f"F1 Score: {synthetic_f1:.4f}")

    # save the base model
    with open('base_topic_classifier.pkl', 'wb') as f:
        pickle.dump(clf, f)
    with open('tfidf_vectorizer.pkl', 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)

    print("Base model saved to 'base_topic_classifier.pkl'")




    # STEP 2: IMPROVED MODEL
    print("\n--- Building Improved Model ---")

    # improved pipeline with feature engineering and better algorithm
    pipeline = Pipeline([
        ('vect', CountVectorizer(ngram_range=(1, 3), min_df=3, max_df=0.9)),
        ('chi2', SelectKBest(chi2, k=2000)),
        ('clf', RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42))
    ])

    # fit improved model
    pipeline.fit(X_train, y_train)

    # Predict on training set
    y_train_pred_improved = pipeline.predict(X_train)
    improved_train_precision = precision_score(y_train, y_train_pred_improved, average='weighted')
    improved_train_recall = recall_score(y_train, y_train_pred_improved, average='weighted')
    improved_train_f1 = f1_score(y_train, y_train_pred_improved, average='weighted')

    print("\nImproved Model - Training Set Metrics:")
    print(f"Precision: {improved_train_precision:.4f}")
    print(f"Recall: {improved_train_recall:.4f}")
    print(f"F1 Score: {improved_train_f1:.4f}")

    # predict on test set
    y_test_pred_improved = pipeline.predict(X_test)
    improved_test_precision = precision_score(y_test, y_test_pred_improved, average='weighted')
    improved_test_recall = recall_score(y_test, y_test_pred_improved, average='weighted')
    improved_test_f1 = f1_score(y_test, y_test_pred_improved, average='weighted')

    print("\nImproved Model - Test Set Metrics:")
    print(f"Precision: {improved_test_precision:.4f}")
    print(f"Recall: {improved_test_recall:.4f}")
    print(f"F1 Score: {improved_test_f1:.4f}")

    # now detailed classification report
    print("\nImproved Model - Classification Report (Test Set):")
    print(classification_report(y_test, y_test_pred_improved))

    # predict on synthetic test set
    y_synthetic_pred_improved = pipeline.predict(X_synthetic)
    improved_synthetic_precision = precision_score(y_synthetic, y_synthetic_pred_improved, average='weighted')
    improved_synthetic_recall = recall_score(y_synthetic, y_synthetic_pred_improved, average='weighted')
    improved_synthetic_f1 = f1_score(y_synthetic, y_synthetic_pred_improved, average='weighted')

    print("\nImproved Model - Synthetic Test Set Metrics:")
    print(f"Precision: {improved_synthetic_precision:.4f}")
    print(f"Recall: {improved_synthetic_recall:.4f}")
    print(f"F1 Score: {improved_synthetic_f1:.4f}")

    # Store improved model metrics
    improved_metrics = {
        'train': {
            'precision': improved_train_precision,
            'recall': improved_train_recall,
            'f1': improved_train_f1
        },
        'test': {
            'precision': improved_test_precision,
            'recall': improved_test_recall,
            'f1': improved_test_f1
        },
        'synthetic': {
            'precision': improved_synthetic_precision,
            'recall': improved_synthetic_recall,
            'f1': improved_synthetic_f1
        }
    }

    # Calculating improvement percentages
    f1_improvement = (improved_metrics['test']['f1'] - base_metrics['test']['f1']) / base_metrics['test']['f1'] * 100
    precision_improvement = (improved_metrics['test']['precision'] - base_metrics['test']['precision']) / base_metrics['test']['precision'] * 100
    recall_improvement = (improved_metrics['test']['recall'] - base_metrics['test']['recall']) / base_metrics['test']['recall'] * 100

    print("\n--- Model Improvement Results ---")
    print(f"F1 Score Improvement: {f1_improvement:.2f}%")
    print(f"Precision Improvement: {precision_improvement:.2f}%")
    print(f"Recall Improvement: {recall_improvement:.2f}%")

    # comparison visualization
    metrics = ['F1 Score', 'Precision', 'Recall']
    base_values = [base_metrics['test']['f1'], base_metrics['test']['precision'], base_metrics['test']['recall']]
    improved_values = [improved_metrics['test']['f1'], improved_metrics['test']['precision'], improved_metrics['test']['recall']]

    plt.figure(figsize=(10, 6))
    x = np.arange(len(metrics))
    width = 0.35

    plt.bar(x - width/2, base_values, width, label='Base Model')
    plt.bar(x + width/2, improved_values, width, label='Improved Model')

    plt.ylabel('Score')
    plt.title('Model Performance Comparison')
    plt.xticks(x, metrics)
    plt.legend()
    plt.tight_layout()
    plt.savefig('model_improvement_comparison.png')
    plt.close()

    # saving the improved model
    with open('improved_topic_classifier.pkl', 'wb') as f:
        pickle.dump(pipeline, f)

    print("\nImproved model saved to 'improved_topic_classifier.pkl'")

if __name__ == "__main__":
    main()

Dataset shape: (590, 3)
                                             Context Training Topic  \
0  Wiping out graft to fatten coffers   Will the ...     Corruption   
1  দূর্নীতির আখড়া গোপালগঞ্জ হাসপাতাল,ধরলো দুদক   ...     Corruption   
2  মুজিব কিল্লা নির্মাণে অর্থ তছরুপে জড়িতদের বিরু...     Corruption   
3  অভিযোগের বিষয়ে জানতে গেলে বোবার অভিনয়...  বিস্...     Corruption   
4  ব্রিজ নির্মাণ হলেও দেয়া হয়নি এপ্রোচ সড়ক, গোপাল...     Corruption   

   Predicted Topic  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN  

Topic distribution:
topic
Mob Justice                   211
Politics                       91
Law and Order                  53
International affairs          49
Religion                       48
Islamic Fundamentalism         24
National Defence               23
Diplomacy                      19
Corruption                     17
Sports                         14
Terrorism                      10
Trade & Commodity Pric

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m


Improved Model - Training Set Metrics:
Precision: 0.9306
Recall: 0.9343
F1 Score: 0.9300

Improved Model - Test Set Metrics:
Precision: 0.6485
Recall: 0.6271
F1 Score: 0.6117

Improved Model - Classification Report (Test Set):
                            precision    recall  f1-score   support

                Corruption       1.00      0.60      0.75         5
                 Diplomacy       1.00      0.67      0.80         3
                  Election       0.00      0.00      0.00         1
Governance & Policy Reform       0.00      0.00      0.00         1
     International affairs       0.67      0.55      0.60        11
    Islamic Fundamentalism       0.25      0.33      0.29         3
             Law and Order       0.88      0.47      0.61        15
               Mob Justice       0.61      0.92      0.73        39
          National Defence       0.00      0.00      0.00         3
                  Politics       0.52      0.50      0.51        22
                  Relig